In [6]:
import pybullet as p
import time
import math
from datetime import datetime
import pybullet_data
import numpy as np
import gym
from gym.utils import seeding
from gym import spaces
import random

pybullet build time: Jun 15 2022 10:51:31


In [2]:
def getinversePoisition(baUid,position_desired,orientation_desired=[]):
    joints_info = []
    joint_ll = [0.0,0.2618,0.0,0.0,0.0,0.1750,0.1750]
    joint_ul = [3.1416,2.8798,3.1416,3.1416,3.1416,1.2741,1.2741]
    useOrientation=len(orientation_desired)
    for i in range(p.getNumJoints(baUid)):
        joints_info.append(p.getJointInfo(baUid, i))
    baEndEffectorIndex = 5 
    numJoints = p.getNumJoints(baUid)
    useNullSpace = 1
    ikSolver = 1
    trailDuration = 30
    pos = [position_desired[0], position_desired[1], position_desired[2]]
    #end effector points down, not up (in case useOrientation==1)
    if useOrientation:
        orn = p.getQuaternionFromEuler([orientation_desired[0],orientation_desired[1] , orientation_desired[2]])
    if (useNullSpace == 1):
      if (useOrientation == 1):
        jointPoses = p.calculateInverseKinematics(baUid, baEndEffectorIndex, pos, orn)
      else:
        jointPoses = p.calculateInverseKinematics(baUid,
                                                  baEndEffectorIndex,
                                                  pos,
                                                  lowerLimits=joint_ll,
                                                  upperLimits=joint_ul,
                                               )
    else:
      if (useOrientation == 1):
        jointPoses = p.calculateInverseKinematics(baUid,
                                                  baEndEffectorIndex,
                                                  pos,
                                                  orn,
                                                  solver=ikSolver,
                                                  maxNumIterations=100,
                                                  residualThreshold=.01)
      else:
        jointPoses = p.calculateInverseKinematics(baUid,
                                                  baEndEffectorIndex,
                                                  pos,
                                                  solver=ikSolver)
    return jointPoses

In [3]:
def get_to_place(robot,position=[0,0,0]):
    orn=[]
    jointPoses = getinversePoisition(robot,position,orn)
    p.resetBasePositionAndOrientation(robot,[0.0, 0.0, 0.0], # position of robot, GREEN IS Y AXIS
                                      [0.000000, 0.000000, 0.000000, 1.000000]) # direction of robot
    #jointPoses = [0.3395072969680578, 0.4057888436625879, 1.1820975801553453, 0.6331240995593741, 0.0, 0.0, 0.0]
    for jointIndex in range(1,p.getNumJoints(robot)): 
        print(jointIndex)
        print(jointPoses[jointIndex-1])
        p.resetJointState(robot, jointIndex, jointPoses[jointIndex-1]) # 1,2,3,4,5,6,7
        p.setJointMotorControl2(bodyIndex=robot,
                                jointIndex=jointIndex,
                                controlMode=p.POSITION_CONTROL,
                                targetPosition=jointPoses[jointIndex-1],
                                targetVelocity=0,
                                force=500,
                                positionGain=0.03,
                                velocityGain=1)


In [4]:
            
def sent_hand_moving(robot,motorCommand):
    baFingerIndexL=6
    baFingerIndexR=7
    left_hand_joint_now = p.getJointState(robot,baFingerIndexL)[0]
    right_hand_joint_now = p.getJointState(robot,baFingerIndexR)[0]
    p.setJointMotorControl2(robot,  #control of fingers
                          baFingerIndexL,
                          p.POSITION_CONTROL,
                          targetPosition=motorCommand+left_hand_joint_now,
                          force=2)

    p.setJointMotorControl2(robot,
                          baFingerIndexR,
                          p.POSITION_CONTROL,
                          targetPosition=motorCommand+right_hand_joint_now,
                          force=2)

In [5]:
def applyAction(robot, motorCommands,baEndEffctorIndex=5): #4 actions
    endEffectorPos = [0, 0, 0]

    dx = motorCommands[0]
    dy = motorCommands[1]
    dz = motorCommands[2]

    fingerAngle = motorCommands[3] 
    state = p.getLinkState(robot, baEndEffctorIndex) # returns 1. center of mass cartesian coordinates, 2. rotation around center of mass in quaternion
    actualEndEffectorPos = state[4] #world position of the link
    endEffectorPos[0] = actualEndEffectorPos[0] + dx
    endEffectorPos[1] =  actualEndEffectorPos[1] +  dy
    endEffectorPos[2] = actualEndEffectorPos[2] +  dz
            
    pos = endEffectorPos
    orn = [0, 0, 0]
    
    jointPoses = getinversePoisition(robot,pos)
    print(jointPoses)
    for i in range(1,baEndEffctorIndex+1): #1,2,3,4,5
        p.resetJointState(robot, i, jointPoses[i-1]) 
        p.setJointMotorControl2(bodyUniqueId=robot,
                                  jointIndex=i,
                                  controlMode=p.POSITION_CONTROL,
                                  targetPosition=jointPoses[i-1],
                                  targetVelocity=0,
                                  force=200,                         
                                  positionGain=0.3,
                                  velocityGain=1)
    sent_hand_moving(robot,fingerAngle)

In [7]:
def reset(robot):
    p.setPhysicsEngineParameter(numSolverIterations=150)
    blockUid=-1
    for i in range(8):
        p.resetJointState(robot, i, 0, 0)
        p.setTimeStep(1./240.)
        # Cube Pos
    for _ in range(100):
        xpos = 0.05 +0.2 * random.random()  # 0.35
        ypos = (random.random() * 0.03) + 0.2  # 0.10 0.50
        zpos = 0.2
        ang = 3.14 * 0.5 + 3.1415925438 * random.random()
        orn = p.getQuaternionFromEuler([0, 0, ang])
        # target Position：
        xpos_target = 0.35 * random.random()  # 0.35
        ypos_target = (random.random() * 0.03) + 0.2  # 0.10 0.50
        zpos_target = 0.2
        ang_target = 3.14 * 0.5 + 3.1415925438 * random.random()
        orn_target = p.getQuaternionFromEuler([0, 0, ang_target])
        dis_between_target_block = math.sqrt(
                (xpos - xpos_target) ** 2 + (ypos - ypos_target) ** 2 + (zpos - zpos_target) ** 2)
        if dis_between_target_block >= 0.1:
            break
    if blockUid == -1:
        blockUid = p.loadURDF("/home/jessie/internship/model/cube.urdf", xpos, ypos, zpos,
                                       orn[0], orn[1], orn[2], orn[3])
        targetUid = p.loadURDF("/home/jessie/internship/model/cube_target.urdf",
                                        [xpos_target, ypos_target, zpos_target],
                                        orn_target, useFixedBase=1)
    else:
        p.removeBody(blockUid)
        p.removeBody(targetUid)
        blockUid = p.loadURDF("/home/jessie/internship/model/cube.urdf", xpos, ypos, zpos,
                                       orn[0], orn[1], orn[2], orn[3])
        targetUid = p.loadURDF("/home/jessie/internship/model/cube_target.urdf",
                                        [xpos_target, ypos_target, zpos_target],
                                        orn_target, useFixedBase=1)
    p.setCollisionFilterPair(targetUid, blockUid, -1, -1, 0)
    p.setGravity(0, 0, -10)
    goal=np.array([xpos_target,ypos_target,zpos_target])
    return goal

In [8]:
def goal_distance(goal_a, goal_b):
    assert goal_a.shape == goal_b.shape
    return np.linalg.norm(goal_a - goal_b, axis=-1)

In [1]:
def compute_reward(achieved_goal, goal, reward_type,distance_threshold):
    # Compute distance between goal and the achieved goal.
    d = goal_distance(achieved_goal, goal)
    if reward_type == 'sparse':
        return -(d > distance_threshold).astype(np.float32)
    else:
        return -d

In [1]:
def set_action(action):
    applyAction(action)

In [9]:
def getObservation(robot):
    observation = []
    # state for gripper(end effector)
    state = p.getLinkState(robot,5)
    pos = state[4]
    pos=list(pos)
    orn = state[5] 
    euler = p.getEulerFromQuaternion(orn)

    observation.extend(list(pos))
    observation.extend(list(euler))

    return observation

In [14]:
def get_obs(robot,block,target):
    end_pos = np.array(getObservation(robot))
    print(end_pos)
    end_pos = end_pos[:3]
    print(end_pos)
    gripperState = p.getLinkState(robot, 5,
                                      computeLinkVelocity=1)
    gripperPos = np.array(gripperState[4])
    gripperOrn_temp = np.array(gripperState[5])
    gripper_linear_Velocity = np.array(gripperState[6])
    gripper_angular_Velocity = np.array(gripperState[7])
    gripperOrn = p.getEulerFromQuaternion(gripperOrn_temp)
    gripperOrn = np.array(gripperOrn)

    blockPos, blockOrn_temp = p.getBasePositionAndOrientation(block)
    blockPos = np.array(blockPos)
    blockOrn = p.getEulerFromQuaternion(blockOrn_temp)
    blockOrn = np.array(blockOrn)

    relative_pos = blockPos - gripperPos

    block_Velocity = p.getBaseVelocity(block)
    block_linear_velocity = np.array(block_Velocity[0])
    block_angular_velocity = np.array(block_Velocity[1])

    target_pos = np.array(p.getBasePositionAndOrientation(target)[0])
    

    print(gripperPos,gripperOrn,gripper_linear_Velocity,gripper_angular_Velocity)

    obs = [
            end_pos.flatten(),
            #gripperPos.flatten(),
            gripperOrn.flatten(),
            gripper_linear_Velocity.flatten(),
            gripper_angular_Velocity.flatten(),
            blockPos.flatten(),
            blockOrn.flatten(),
            relative_pos.flatten(),
            #relative_orn.flatten(),
            target_pos.flatten(),
            #target_relative_pos.flatten()
            block_linear_velocity.flatten(),
            block_angular_velocity.flatten(),
        ]
    achieved_goal = blockPos.copy()
    
    

In [ ]:
def step(robot, action,goal):
    action = np.clip(action,-0.5,0.5)
    action[3]=0
 
    set_action(action)
    # print(action[3])
    #一个动作执行20个仿真步
    for _ in range(20):
        p.stepSimulation()
    obs = get_obs()
    done = False

    reward = compute_reward(obs['achieved_goal'], goal, reward_type="sparse",distance_threshold=0.001)
    return obs, reward, done

In [13]:
physicsClient = p.connect(p.DIRECT)#
p.resetSimulation()
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-10)
planeId = p.loadURDF("plane.urdf")
robot = p.loadURDF("/home/jessie/internship/model/braccio_arm_clean.urdf")
reset(robot)
get_obs(robot)
#applyAction(robot,[0.8,0.5,0.1,1.2570])
#get_to_place(robot)


[-2.32830644e-10  6.29990846e-02 -5.29997982e-02  0.00000000e+00
 -1.57079633e+00  1.57079263e+00]
[-2.32830644e-10  6.29990846e-02 -5.29997982e-02]
[-2.32830644e-10  6.29990846e-02 -5.29997982e-02] [ 0.         -1.57079633  1.57079263] [0. 0. 0.] [0. 0. 0.]


In [ ]:
class barobotGymEnv(gym.Env):
    def __init__(self,
                 model_path,
                 n_substeps,
                 gripper_extra_height,
                 block_gripper,
                 has_object,
                 target_in_the_air,
                 target_offset,
                 obj_range,
                 target_range,
                 distance_threshold,
                 initial_qpos,
                 reward_type):
        IS_USEGUI = 1
        self.gripper_extra_height = gripper_extra_height
        self. block_graipper = block_gripper
        self. has_object = has_object
        self. target_in_the_air = target_in_the_air
        self.target_offset = target_offset
        self.obj_range = obj_range
        self.target_range = target_range
        self.distance_threshold = distance_threshold
        self.reward_type = reward_type
        self.model_path=model_path
        self.n_substeps=n_substeps
        self. n_action = 4
        self. blockUid = -1
        self. initial_qpos = initial_qpos
        self._urdfRoot = pybullet_data.getDataPath()
        self.seed()
        if IS_USEGUI:
            self.physics = p.connect(p.GUI)
        else:
            self.physics = p.connect(p.DIRECT)
        #load robot 
        self._barobot = robot
        self._timeStep = 1./240.
        action_dim = 4
        self._action_bound = 0.5
        action_high = np.array([self._action_bound] * action_dim)
        self.action_space = spaces.Box(-action_high, action_high)
        #reset the environment
        self.reset()
    def compute_reward(self,achieved_goal,goal,info):
        #compute the distance between goal and the achieved goal
        d =goal_distance(achieved_goal,goal)
        if self.reward_type == "sparse":
            return -(d>self.distance_threshold).astype(np.float32)
        else:
            return -d  

    def step(self,action):
        action = np.clip(action,-0.5,0.5)
        if p.getClosestPoints(self._barobot,self.blockUid,0.0001):
            action[3]=-1
        self._set_action(action)
        for _ in range(self.n_substeps):
            p.stepSimulation()
        obs = self._get_obs()
        done = False
        info = {
            'is_success': self._is_success(obs['achieved_goal'], self.goal),
        }
        reward = self.compute_reward(obs["achieved_goal"],self.goal)
        return obs,reward,done,info
    
    def reset(self):
        p.setPhysicsEnginePArameter(numSolverIterations=150)
        for i in range(8):
            p.resetJointState(self._barobot,i,0,0)
        p.setTimeSTep(self._timeStep)
        #cube pos
        for _ in range(100):
            xpos = 0.15 +0.2 * random.random()  # 0.35
            ypos = (random.random() * 0.3) + 0.2  # 0.10 0.50
            zpos = 0.2
            ang = 3.14 * 0.5 + 3.1415925438 * random.random()
            orn = p.getQuaternionFromEuler([0, 0, ang])
    def seed():
        pass
    def render():
        pass